# **Anthropic/Claude-3-sonnet**

# **Step 1: Libraries**

In [1]:
!pip install anthropic --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 850.5/850.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [50]:
from tqdm import tqdm
import time
import pandas as pd
import anthropic

# **Step 2: Load Data**

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df = pd.read_csv('/content/drive/MyDrive/Medical_Transcription_Extraction/mtsamples.csv')

In [10]:
transcription_list = df['transcription'].to_list()

# **Step 3: Model**

**To get Claude API token, click [here](https://console.anthropic.com/settings/plans)**

In [17]:
from getpass import getpass
claude_token = getpass('Enter your Claude API key: ')

Enter your Claude API key: ··········


In [18]:
client = anthropic.Anthropic(api_key=claude_token)

# **Step 4: Inference**

In [75]:
def get_chatglm_response(transcription):
    prompt = f"This is a patient's transcription: '{transcription}' \n\n Please use this format as the answer: The patient age is [Your extracted answer]. The patient will get [Your extracted answer] as treatment. Note: If the treatment has already been administered, it's also considered as the treatment, the format will be revised slightly: The patient got [Your extracted answer] as treatment. If there is no specific neither age or treatment , you can return unknown, unknown. If there is no specific age but treatment, you can return unknown, The patient will get [Your extracted answer] as treatment. There may be not only one treatment"

    message = client.messages.create(
    model="claude-3-sonnet-20240229", # claude 3 Sonnet
    max_tokens=1000,
    temperature=0.0,
    system="follow my instruction format, don't answer other words",
    messages=[
        {"role": "user", "content": f"{prompt}"}
    ]
)
    return message.content[0].text

In [46]:
res = []

In [76]:
for transcription in tqdm(transcription_list[491:]):
    response = get_chatglm_response(transcription)
    res.append(response)
    print(response)
    time.sleep(10)

  0%|          | 0/4508 [00:03<?, ?it/s]


RateLimitError: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of request tokens has exceeded your rate limit (https://docs.anthropic.com/claude/reference/rate-limits). Please reduce the the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}}

In [78]:
pd.DataFrame(res).to_csv('/content/drive/MyDrive/Medical_Transcription_Extraction/res_claude-3-sonnet.csv')